###### <img src="Electronic_Brain.png" width="200" style="float:left">
<h1> Summer 2021 ML Course.</h1>
<h2> Exercise 4: Cython version(s) of Ronny Roshbakir's trading algorithm.</h2>

In [ ]:
import time
import numpy as np

%load_ext Cython
%load_ext line_profiler

# Home-grown scripts & libraries.
from ex4_utils import py_naive_profit

In [ ]:
# Create a semi-realistic prices array.
# Start off with a mostly NaN array with a few 'turning points' (local min/max).
prices = np.full(5000, fill_value=np.nan)
prices[[0, 1250, 3000, -1]] = [80., 30., 75., 50.]

# Linearly interpolate the missing values and add some noise.
# NOTICE how the turning (valid) points are selected and all others are interpolated.
x = np.arange(len(prices))
is_valid = ~np.isnan(prices)                                  # Only look at valid numbers.
prices = np.interp(x=x, xp=x[is_valid], fp=prices[is_valid])  # Interpolate between them.
prices += np.random.randn(len(prices)) * 2                    # Add normally distributed noise.

In [ ]:
%lprun -f py_naive_profit py_naive_profit(prices)

<img src="desktop-computer-icon.png" width="90" style="float:left; margin-right: 10px;">
<img src="Roni_Roshbakir.png" width="36" style="float:left; margin-right: 10px;">
</br>Please Help Roni perform the brute-force max-profit calculation.</br>

In [ ]:
%%cython -a
# cython: profile=True
import time
import numpy as np
cimport numpy as np
cimport cython

# Re-implement the price generator, this time in Cython (actually it's still Numpy).
cdef generate_stock_prices():

    # Create a semi-realistic prices array.
    # Start off with a mostly NaN array with a few 'turning points' (local min/max).
    prices = np.full(5000, fill_value=np.nan)
    prices[[0, 1250, 3000, -1]] = [80., 30., 75., 50.]

    # Linearly interpolate the missing values and add some noise.
    # NOTICE how the turning (valid) points are selected and all others are interpolated.
    x = np.arange(len(prices))
    is_valid = ~np.isnan(prices)                                  # Only look at valid numbers.
    prices = np.interp(x=x, xp=x[is_valid], fp=prices[is_valid])  # Interpolate between them.
    prices += np.random.randn(len(prices)) * 2                    # Add normally distributed noise.

    return(prices)

"""
Implement a Cython version of Ronny's naive algorithm.
NOTICE: a numpy "float" is actually a C "double".
WARNING: this is a C function - nothing Pythonic will work inside!
WARNING: make sure to type each variable inside, otherwise things will be SLOW.
NOTICE: the Numpy array is passed into this C function as is.
NOTICE: the "double[::1]" declaration means we guarantee the input is a contiguous array,
        which (by default) Numpy arrays are (In the 2D case, write "double[:,::1]").
"""
cdef double cy_naive_profit(double[::1] prices):
    cdef double max_profit = -1.0
    cdef int i,j
    for i in range(len(prices)):
        for j in range(i):
            if prices[i] - prices[j] > max_profit:
                max_profit = prices[i] - prices[j]
    return max_profit

prices = generate_stock_prices()
time1 = time.perf_counter()
cy_profit = cy_naive_profit(prices)
time2 = time.perf_counter()
print("Cython naive profit:", cy_profit, "time:", time2 - time1)

<img src="desktop-computer-icon.png" width="90" style="float:left; margin-right: 10px;">
<img src="Roni_Roshbakir.png" width="36" style="float:left; margin-right: 10px;">
</br>DANGER ZONE!!! Help Roni perform the brute-force max-profit calculation with array bounds checking disabled.</br>

In [ ]:
%%cython -a
import time
import numpy as np
cimport numpy as np
cimport cython

cdef generate_stock_prices():
    prices = np.full(5000, fill_value=np.nan)
    prices[[0, 1250, 3000, -1]] = [80., 30., 75., 50.]
    x = np.arange(len(prices))
    is_valid = ~np.isnan(prices)                                  # Only look at valid numbers.
    prices = np.interp(x=x, xp=x[is_valid], fp=prices[is_valid])  # Interpolate between them.
    prices += np.random.randn(len(prices)) * 2                    # Add normally distributed noise.
    return(prices)

"""
Implement a version of Ronny's naive algorithm with array bounds checking disabled.
SUPER WARNING: disabling array bounds checks is DANGEROUS!
               Only do this *AFTER* the code as been *FULLY* DEBUGGED!!!
               Don't say I didn't warn you!
"""
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef double cy_naive_profit_no_bounds(double[::1] prices):
    cdef double max_profit = -1.0
    cdef int i,j
    for i in range(len(prices)):
        for j in range(i):
            if prices[i] - prices[j] > max_profit:
                max_profit = prices[i] - prices[j]
    return max_profit

prices = generate_stock_prices()
time1 = time.perf_counter()
no_bounds_profit = cy_naive_profit_no_bounds(prices)
time2 = time.perf_counter()
print("No bounds-checking profit:", no_bounds_profit, "time:", time2 - time1)

<img src="desktop-computer-icon.png" width="90" style="float:left; margin-right: 10px;">
<img src="Roni_Roshbakir.png" width="36" style="float:left; margin-right: 10px;">
</br>Please help Roni perform brute-force max-profit calculation in parallel.</br>

In [ ]:
%%cython -a
import time
import numpy as np
cimport numpy as np
cimport cython
from cython.parallel import parallel, prange

cdef generate_stock_prices():
    prices = np.full(5000, fill_value=np.nan)
    prices[[0, 1250, 3000, -1]] = [80., 30., 75., 50.]
    x = np.arange(len(prices))
    is_valid = ~np.isnan(prices)                                  # Only look at valid numbers.
    prices = np.interp(x=x, xp=x[is_valid], fp=prices[is_valid])  # Interpolate between them.
    prices += np.random.randn(len(prices)) * 2                    # Add normally distributed noise.
    return(prices)

"""
Implement *only* the (naive algorithm's) INNER loop.
NOTE: the nogil keyword does NOT release Python's infamous GIL, it only
instructs Cython that the function *may* be called without the GIL.
"""
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef double inner_loop_max_profit(int i, double[::1] prices) nogil:
    cdef double max_profit = -1.0
    cdef double final_price = prices[i]
    cdef int j
    for j in range(i):
        if final_price - prices[j] > max_profit:
            max_profit = final_price - prices[j]
    return(max_profit)

"""
"""
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef cy_naive_profit_parallel(numpy_prices):
    # Make sure to declare types for each and every variable.
    cdef int i
    cdef int len_prices = len(numpy_prices)
    cdef int num_cpus = 8
    cdef double global_max_profit = 0.0

    # Set up memoryviews to hold the maximal profits possible up to each price index.
    # NOTE: the "double[::1]" means we're declaring a contiguous array.
    cdef double[::1] max_profits = np.zeros(len(numpy_prices))
    cdef double[::1] memview_prices = numpy_prices

    # In order to run loops in parallel, we instruct Python to disable the GIL (Global Interpreter Lock).
    # We also provide the number of threads we want to run in parallel.
    # In this case, we define one thread per core (note that some CPUs can run two threads per core).
    with nogil, parallel(num_threads=num_cpus):

        # NOTE: inside this loop, nothing Pythonic will work (as the GIL is disabled).
        # NOTE: we use prange() to implement the OUTER loop in parallel.
        # for i in prange(len_prices, schedule='guided'):
        for i in prange(len_prices):
            max_profits[i] = inner_loop_max_profit(i, memview_prices)

    # With max_profits populated, simply iterate to find the max.
    for i in range(len(max_profits)):
        if max_profits[i] > global_max_profit:
            global_max_profit = max_profits[i]

    return(global_max_profit)

prices = generate_stock_prices()
time1 = time.perf_counter()
super_ronny_profit = cy_naive_profit_parallel(prices)
time2 = time.perf_counter()
print("Cython naive profit:", super_ronny_profit, "time:", time2 - time1)

<img src="desktop-computer-icon.png" width="90" style="float:left; margin-right: 10px;">
<img src="Batya_Bingo.png" width="80" style="float:left; margin-right: 10px;">
</br>Please construct an example where the parallel computation is faster than the serial.</br>